In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from thorlabs_elliptec import ELLx, ELLError, ELLStatus, list_devices,find_device

In [35]:
def AnglepairFunction(thetain):
    thetaOut = np.degrees(0.5*np.arccos(np.sqrt(0.5 * (1 - np.sqrt(np.sin(2 * np.radians(thetain)))))))
    return thetaOut
def SetAnglePair(stage1,stage2,AngleSet):
    stage1.move_absolute(AngleSet,blocking=True)
    AnglePair=AnglepairFunction(AngleSet)
    stage2.move_absolute(AnglePair,blocking=True)
    print(AngleSet,stage1.get_position())
    print(AnglePair,stage2.get_position())

In [4]:
print(list_devices())
# device=/dev/cu.usbserial-DT04N7X4, manufacturer=FTDI, product=FT230X Basic UART, vid=0x0403, pid=0x6015, serial_number=DT04N7X4, location=1-1.2.1
# device=/dev/cu.usbserial-DT04N7PW, manufacturer=FTDI, product=FT230X Basic UART, vid=0x0403, pid=0x6015, serial_number=DT04N7PW, location=0-1.3.3

device=/dev/cu.wlan-debug, manufacturer=None, product=None, vid=None, pid=None, serial_number=None, location=None
device=/dev/cu.Bluetooth-Incoming-Port, manufacturer=None, product=None, vid=None, pid=None, serial_number=None, location=None
device=/dev/cu.usbserial-DT04N7X4, manufacturer=FTDI, product=FT230X Basic UART, vid=0x0403, pid=0x6015, serial_number=DT04N7X4, location=1-1.3


# Connect to mounts
### So some time the board will need to be reset using thorlabs software as that software only allows one COM port communication and it does so funny thing with alisis the other mount. you just factor reset to fix.

In [32]:
# stage = ELLx(serial_port='/dev/cu.usbserial-DT04N7X4',vid=0x0403,pid=0x6015,serial_number='DT04N7PW', location='0-1.3.3',manufacturer='FTDI', product='FT230X Basic UART')
stages=[]
stages.append( ELLx(serial_port='/dev/cu.usbserial-DT04N7X4'))
stages.append(ELLx(serial_port='/dev/cu.usbserial-DT04N7PW'))

stageCount=len(stages)
for istage in range(stageCount):
    print(f"{stages[istage].model_number} #{stages[istage].device_id} on {stages[istage].port_name}, serial number {stages[istage].serial_number}, status {stages[istage].status.description}")

    

ELL14/M #0 on /dev/cu.usbserial-DT04N7X4, serial number 11400790, status ok
ELL14/M #0 on /dev/cu.usbserial-DT04N7PW, serial number 11400791, status unknown


# Home both mounts

In [33]:
# Move device to the home position
for istage in range(stageCount):
    print(istage,stages[istage].get_position())
    stages[istage].home(blocking=True)
    print(istage,stages[istage].get_position())


0 -24.037
0 0.0
1 -24.029
1 -0.005


In [42]:
SetAnglePair(stages[0],stages[1],0)


0 -0.01
22.5 22.495


Error reading response string! (requested '0gs', received '')
Error reading response string! (requested '0gs', received '')
Could not query device status! (response was '')
Could not query device status! (response was '')
Exception in callback ELLx._update_status()
handle: <TimerHandle when=17669.881672917 ELLx._update_status()>
Traceback (most recent call last):
  File "/Users/s4356803/opt/anaconda3/lib/python3.9/site-packages/serial/serialposix.py", line 621, in write
    n = os.write(self.fd, d)
OSError: [Errno 6] Device not configured

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/s4356803/opt/anaconda3/lib/python3.9/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/Users/s4356803/opt/anaconda3/lib/python3.9/site-packages/thorlabs_elliptec/__init__.py", line 426, in _update_status
    reply_data = self._write_command("gp")
  File "/Users/s4356803/opt/anaconda3/lib

In [29]:
angleCount=5
# angles_arr=np.linspace(0,360-0.0001,angleCount)
angles_arr=np.linspace(360-0.0001,0,angleCount)

angle_measured_arr=np.zeros(angleCount)

counts=20
for iCount in range(counts):
    for iangle in range(angleCount):
        
        input_angle = angles_arr[iangle] + 30 * np.sin(np.radians(angles_arr[iangle]))
        stages[0].move_absolute(input_angle,blocking=True)
        stages[1].move_absolute(input_angle,blocking=True)
        
        time.sleep(0.1)
        
        angle_measured=stages[0].get_position()
        angle_measured_arr[iangle]=angle_measured
        print(angle_measured-angles_arr[iangle])

plt.plot(angles_arr)
plt.plot(angle_measured_arr)

-0.002900000000011005
-29.970925000000022
0.003049999999973352
29.998025
0.008
9.999999997489795e-05
-29.965925000000027
0.0050499999999829015
29.998025
0.015
-0.017899999999997362
-29.96092500000003
0.003049999999973352
29.99602499999999
0.003
0.0030999999999608008
-29.965925000000027
-0.009950000000003456
29.99602499999999
0.008
-0.004900000000020555
-29.965925000000027
4.999999998744897e-05
30.001025
0.033
-0.002900000000011005
-29.967925000000008
0.008049999999997226
30.001025
-0.008
-0.007900000000006457
-30.00592500000002
0.0050499999999829015
29.99602499999999
0.025
-0.007900000000006457
-29.99792500000001
0.0050499999999829015
29.99602499999999
0.023
0.0030999999999608008
-30.002925000000005
-0.002949999999998454
29.998025
0.02
-0.004900000000020555
-29.990925000000033
-0.0049500000000080036
29.99602499999999
0.03
9.999999997489795e-05
-29.995925000000028
-0.002949999999998454
30.003024999999994
0.028
-0.007900000000006457
-29.995925000000028
-0.002949999999998454
30.0030249999

KeyboardInterrupt: 

In [30]:

for istage in range(stageCount):
    stages[istage].close()

In [20]:
# Move device to the home position
stage.home()
# Movements are in real units appropriate for the device (degrees, mm).
stage.move_absolute(45.0)
# The raw device encoder units can also be used with the move raw variants.
stage.move_absolute_raw(23456)

# By default, move commands are asynchronous (non-blocking) and return immediately,
# but you can manually wait for it to be in position
stage.move_absolute(45.0)
stage.wait()
stage.move_relative(-12.34)
# or test whether movement is still in progress.
print(stage.is_moving())
stage.wait()
print(stage.is_moving())
print(f"{stage.get_position()}{stage.units}")
# Prints something like:
# True
# False
# 32.655°

# Synchronous behaviour can also be achieved by setting the blocking=True parameter,
# which will perform the wait before returning from each movement command.
stage.home(blocking=True)
stage.move_absolute(1.23, blocking=True)
stage.move_relative(-0.98, blocking=True)

# When using the synchronous behaviour, any error during movement will raise an exception.
try:
    stage.move_absolute(-9999)
except ELLError as ex:
    if ex.status == ELLStatus.OUT_OF_RANGE:
        # Requested move beyond device limits
        print("Device can't move there!")
    else:
        # Other error, eg stage held or blocked so it can't move
        print(f"Movement error: {ex}")
else:
    print("Move completed OK")

# When using asynchronous calls, any errors won't have been detected yet,
# so instead, the is_moving() and wait() methods can raise the exception instead.
stage.move_relative(300)
try:
    print(stage.is_moving(raise_errors=True))
    stage.wait(raise_errors=True)
    print(stage.is_moving(raise_errors=True))
except ELLError as ex:
    print(f"Movement error: {ex}")

# Once done with the device, it can be specifically closed. Commands to the stage will no
# longer work until the device is re-initialised.
stage.close()

True
False
32.645°
Move completed OK
True
False
